## Eden Duktzayev 316365733

In [419]:
import requests
import json
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [420]:
api_key='12345678'#fake

### Reading from the data

In [421]:
datafile = "C:/Users/edend/לימודים אריאל/תעשיה וניהול שנה ב/סמסטר ב/מבוא לידע ונתונים/מטלות/destinations_LP_crawler_Ex5.csv"

In [422]:
data = pd.read_csv((datafile),encoding='ISO-8859-1')
data

,city,city_LP
0,Antalya,mediterranean-coast/antalya
1,Athens,Athens
2,Ayia-napa,the-republic-of-cyprus/Agia-napa
3,Bangkok,Bangkok
4,Barcelona,Barcelona
...,...,...
71,Venice,Venice
72,Vienna,Vienna
73,Warsaw,Warsaw
74,Zakynthos,Zakynthos


### Create a new column named Country

In [428]:
data['country']="yes"
data

,city,city_LP,country
0,Antalya,mediterranean-coast/antalya,yes
1,Athens,Athens,yes
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,yes
3,Bangkok,Bangkok,yes
4,Barcelona,Barcelona,yes
...,...,...,...
71,Venice,Venice,yes
72,Vienna,Vienna,yes
73,Warsaw,Warsaw,yes
74,Zakynthos,Zakynthos,yes


### Data correction - Krakow and Kos

In [424]:
for i in range(len(data)):
    if '¿' in data.loc[i,'city'] or 'ï' in data.loc[i,'city'] or 'Kos' in data.loc[i,'city'] :
        data.loc[i,'city']=data.loc[i,'city'].replace('ï', 'o')
        data.loc[i,'city']=data.loc[i,'city'].replace('¿', 'w')
        data.loc[i,'city']=data.loc[i,'city'].replace('Kos', 'Κως')

In [429]:
data

,city,city_LP,country
0,Antalya,mediterranean-coast/antalya,yes
1,Athens,Athens,yes
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,yes
3,Bangkok,Bangkok,yes
4,Barcelona,Barcelona,yes
...,...,...,...
71,Venice,Venice,yes
72,Vienna,Vienna,yes
73,Warsaw,Warsaw,yes
74,Zakynthos,Zakynthos,yes


### using api geocode

In [430]:
for i in range(len(data)):
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s&key=%s" % (data['city'][i],api_key)
    try:
        respon = requests.get(url)
        if not respon.status_code == 200:
            print("HTTP error",respon.status_code)
        else:
            try:
                respon = requests.get(url).json()
            except:
                print("Response not in valid JSON format")
        country=respon['results'][0]['formatted_address'].split(',')[-1]
        country=country.split(' ')[-1]
        for n in country:
            if n.isdigit():
                country=respon['results'][0]['formatted_address'].split(',')[-2]
                country=country.split(' ')[-1]
        data['country'][i]=country
    except:
        print("Something went wrong with requests.get")

In [431]:
data.head()

,city,city_LP,country
0,Antalya,mediterranean-coast/antalya,Turkey
1,Athens,Athens,Greece
2,Ayia-napa,the-republic-of-cyprus/Agia-napa,Cyprus
3,Bangkok,Bangkok,Thailand
4,Barcelona,Barcelona,Spain


### Creating a dictionary that contain 3 columns

In [432]:
LP_destinations={'city':[],'country':[],'description':[]}
type(LP_destinations)

dict

### Using BeautifulSoup to get html text

In [433]:
mylist=[]
for x in range(len(data)):
    url2="https://www.lonelyplanet.com/"+data['country'][x]+'/'+data['city_LP'][x]
    response = requests.get(url2)
    if not response.status_code == 200:
        print("HTTP error",response.status_code)
    soup = BeautifulSoup(response.content,'html.parser')
    mylist=[]
    for k in soup.find('section',{'class':"text-black-400"}):
        mylist.append(k.get_text())
    LP_destinations['city'].append(data['city'][x])
    LP_destinations['country'].append(data['country'][x])
    LP_destinations['description'].append(mylist[0])
print('Done!')

Done!


### Beyond DataFrame

In [434]:
LP_destinations=pd.DataFrame(LP_destinations)

### Create a CSV file

In [435]:
LP_destinations.to_csv('C:/Users/edend/OneDrive/שולחן העבודה/LP_destinations.csv',index=False)

### Reading about Barcelona from the file

In [436]:
new_file=pd.read_csv('C:/Users/edend/OneDrive/שולחן העבודה/LP_destinations.csv')
Barcelona=new_file.loc[new_file['city']=='Barcelona']
Barcelona.reset_index(drop=True, inplace=True)
pd.DataFrame(Barcelona,columns=['city','country','description'])

,city,country,description
0,Barcelona,Spain,Barcelona is an enchanting seaside city with b...


### An explanation of Barcelona

In [438]:
print(Barcelona.loc[0,'description'])

Barcelona is an enchanting seaside city with boundless culture, fabled architecture and a world-class drinking and dining scene.
Architecture of the Ages
Barcelona's architectural treasures span 2000-plus years. Towering temple columns, ancient city walls and subterranean stone corridors provide a window into Roman-era Barcino. Fast forward a thousand years or so to the Middle Ages by taking a stroll through the shadowy lanes of the Gothic quarter, past tranquil plazas and soaring 14th-century cathedrals. In other parts of town bloom the sculptural masterpieces of Modernisme, a mix of ingenious and whimsical creations by Gaudí and his Catalan architectural contemporaries. Barcelona has also long inspired artists, including Pablo Picasso and Joan Miró, both of whom have major Barcelona museums devoted to their works.
A Moveable Feast
The masters of molecular gastronomy – Albert Adrià, Carles Abellan et al – are part of the long and celebrated tradition of Catalan cooking. Simple, flavou